# Load sales data

In [26]:
import pandas as pd
import numpy as np
from itertools import product
sales = pd.read_csv('./data-readonly/sales_train.csv.gz')
cur_shops = sales['shop_id'].unique()
items = pd.read_csv('./data-readonly/items.csv')
cur_items = items['item_id'].unique()
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
df_All_train = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
df_All_train.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
df_All_train = df_All_train.to_sparse()

C:\Users\Xiang\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [29]:
df_All_train.reset_index(drop=True)
df_All_train.to_hdf('All_train.hdf',key='train')

# Build a sparse dataframe for all items, all shops, all months

In [7]:
all_data.describe()

,shop_id,item_id,date_block_num,target
count,1.091385e+07,1.091385e+07,1.091385e+07,1.091385e+07
mean,3.118720e+01,1.130926e+04,1.497334e+01,3.342731e-01
std,1.734959e+01,6.209978e+03,9.495618e+00,3.417243e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01
25%,1.600000e+01,5.976000e+03,7.000000e+00,0.000000e+00
50%,3.000000e+01,1.139100e+04,1.400000e+01,0.000000e+00
75%,4.600000e+01,1.660500e+04,2.300000e+01,0.000000e+00
max,5.900000e+01,2.216900e+04,3.300000e+01,2.253000e+03


In [24]:
sales['item_id'].unique().shape

(21807,)

In [25]:
fpath = "./data-readonly/items.csv"
df_Items = pd.read_csv(fpath)
df_Items['item_id'].unique().shape

(22170,)